In [1]:
#import external libraries
import pandas as pd
import os
import numpy as np
import datetime
import pytz
import matplotlib.pyplot as plt
import calendar
from statsmodels.formula.api import ols

#plotting
%matplotlib notebook
import seaborn as sns
sns.set_style("whitegrid")
import plot_funcs

#import my settings
from settings import Glacier, Station, base_path
import CleanWxData as wx

In [2]:
save_plots=True
save_dir='./figs/'+Glacier+Station+ '/trends/'

In [3]:
#Read in data
data_dir=base_path +"Data/"+Glacier+ r"/AllYears/Wx/LVL3/"
fl="Undercatch_Adj" + Glacier+ "_Daily_Weather.csv"
if Glacier =='SouthCascade':
    fl="Input_SouthCascade_Daily_Weather.csv"
pth=os.path.join(data_dir, fl)
dat=pd.read_csv(pth)
print("read data from "+ pth)

#Set time index
dat.Date=pd.to_datetime(dat.Date, infer_datetime_format=True)
dat=dat.set_index('Date')

#Abbreviate precip column name
dat.rename(columns={'Precip_MeasuredWindSpeed_UndercatchAdj':'precip_adj'}, inplace=True)
#Add month column
dat['month']=dat.index.month

read data from Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana/AllYears/Wx/LVL3/Undercatch_AdjGulkana_Daily_Weather.csv


In [4]:
#drop rows where both precip and temp are null; neccesary for glaciers that may have many leading or trailing NANs
#dat=dat[['Precipitation', 'Temperature']].dropna(how='all')

In [5]:
dat['month']=dat.index.month

In [6]:
#Define Seasons
summer_months=[6,7,8]
winter_months=[12,1,2]
spring_months=[3,4,5]
fall_months=[9,10,11]
season_list=[summer_months, winter_months, spring_months, fall_months]

In [7]:
#TOMORROW - Do the stuff in Biennek 2017! Woop Woop!

#first - need to read in daily high and low temps from: 
#"Q:\Project Data\GlacierData\Benchmark_Program\Data\Wolverine\AllYears\Wx\LVL2\emily\wolverine990_daily_LVL2.csv"

In [8]:
startdate_minmax=dat.Temperature.first_valid_index()

In [9]:
dat['PrecipR3']=dat.Precipitation.rolling(3, center=True).mean() #3 day moving window avg.

In [10]:
dat['temp_anomaly']=(dat.Temperature -dat.Temperature.mean())/dat.Temperature.std()

In [11]:
dat_overall_highest_temps=dat[dat.index.isin(dat.Temperature.nlargest(10).index)].copy()

In [12]:
dat_overall_lowest_temps=dat[dat.index.isin(dat.Temperature.nsmallest(10).index)].copy()

In [13]:
#Dataframe of all the hottest temperatures on record for each season
hot_temp_dat=pd.DataFrame()
for season in season_list:
    print(season)
    season_dat=dat[dat.month.isin(season)]
    dat2=season_dat[season_dat.index.isin(season_dat.Temperature.nlargest(10).index)].copy()
    if season==[12,1,2]:
        dat2['season']='Winter'
    if season==[9,10,11]:
        dat2['season']='Fall'
    if season==[3,4,5]:
        dat2['season']='Spring'
    if season==[6,7,8]:
        dat2['season']='Summer'
    hot_temp_dat=hot_temp_dat.append(dat2)

[6, 7, 8]
[12, 1, 2]
[3, 4, 5]
[9, 10, 11]


In [14]:
#Dataframe of all the COLDEST temperatures on record for each season
cold_temp_dat=pd.DataFrame()
for season in season_list:
    print(season)
    season_dat=dat[dat.month.isin(season)]
    dat2=season_dat[season_dat.index.isin(season_dat.Temperature.nsmallest(10).index)].copy()
    if season==[12,1,2]:
        dat2['season']='Winter'
    if season==[9,10,11]:
        dat2['season']='Fall'
    if season==[3,4,5]:
        dat2['season']='Spring'
    if season==[6,7,8]:
        dat2['season']='Summer'
    cold_temp_dat=cold_temp_dat.append(dat2)

[6, 7, 8]
[12, 1, 2]
[3, 4, 5]
[9, 10, 11]


In [15]:
#Dataframe of all the HIGHEST PRECIP on record for each season (on 3-day smoothed precip events)
high_precip_dat=pd.DataFrame()
for season in season_list:
    print(season)
    season_dat=dat[dat.month.isin(season)]
    dat2=season_dat[season_dat.index.isin(season_dat.PrecipR3.nlargest(10).index)].copy()
    if season==[12,1,2]:
        dat2['season']='Winter'
    if season==[9,10,11]:
        dat2['season']='Fall'
    if season==[3,4,5]:
        dat2['season']='Spring'
    if season==[6,7,8]:
        dat2['season']='Summer'
    high_precip_dat=high_precip_dat.append(dat2)

[6, 7, 8]
[12, 1, 2]
[3, 4, 5]
[9, 10, 11]


In [16]:
# #Dataframe of all precip (non-trace) split by phase
# phase_freq_dat=pd.DataFrame()
# for season in season_list:
#     print(season)
#     season_dat=dat[dat.month.isin(season)]
#     dat2=season_dat[season_dat.index.isin(season_dat[season_dat.Precipitation>3].index)].copy()
#     if season==[12,1,2]:
#         dat2['season']='Winter'
#     if season==[9,10,11]:
#         dat2['season']='Fall'
#     if season==[3,4,5]:
#         dat2['season']='Spring'
#     if season==[6,7,8]:
#         dat2['season']='Summer'
#     phase_freq_dat=phase_freq_dat.append(dat2)

In [17]:
def define_decades(dat2, possible_decades):
    decades=pd.Series(dat2.index.year).astype('str').str.slice(0,3) +'0'
    dat2['decades']=decades.values
    dat2['decades']=pd.Categorical(decades, categories=possible_decades)    
    return(dat2)

In [18]:
if (Glacier + Station == ('Wolverine990') )| (Glacier + Station==('Gulkana1480')):
    decade_list=['1970', '1980', '1990', '2000', '2010']

In [19]:
if Glacier + Station == ('Sperry2440'):
    decade_list=['2000', '2010']

In [20]:
if Glacier+Station=='SouthCascade270':
        decade_list=['1950', '1960', '1970', '1980', '1990', '2000', '2010']

In [21]:
if Glacier+Station=='LemonCreek5':
        decade_list=['1950', '1960', '1970', '1980', '1990', '2000', '2010']

In [22]:
Glacier+Station

'Gulkana1480'

In [23]:
dat.Precipitation.plot()

<IPython.core.display.Javascript object>

In [24]:
#Create categorical variable for decade
hot_temp_dat=define_decades(hot_temp_dat, decade_list)
cold_temp_dat=define_decades(cold_temp_dat, decade_list)
high_precip_dat=define_decades(high_precip_dat, decade_list)
dat_overall_highest_temps=define_decades(dat_overall_highest_temps, decade_list)
dat_overall_lowest_temps=define_decades(dat_overall_lowest_temps, decade_list)

In [25]:
hot_temp_dat['season']=pd.Categorical(hot_temp_dat.season, categories=['Winter', 'Fall', 'Spring', 'Summer'], ordered=True)
cold_temp_dat['season']=pd.Categorical(cold_temp_dat.season, categories=['Winter', 'Fall', 'Spring', 'Summer'], ordered=True)
high_precip_dat['season']=pd.Categorical(high_precip_dat.season, categories=['Winter', 'Fall', 'Spring', 'Summer'], ordered=True)

In [26]:
#dat[: first_bin_year]

In [27]:
pre_first_bin_year=str(pd.to_numeric(decade_list[0])-1)
extra_years_in_first_bin=pd.Series(dat[str(dat.Temperature.first_valid_index().year): pre_first_bin_year].index.year).unique()
n_extra_yrs_first_bin=len(extra_years_in_first_bin)

In [28]:
n_extra_yrs_first_bin

3

In [29]:
post_last_bin_year=str(pd.to_numeric(decade_list[-1])+1)

In [30]:
n_years_in_last_quasi_decade=len(pd.Series(dat[post_last_bin_year:].index.year).unique())

In [31]:
pd.Series(dat[post_last_bin_year:].index.year).unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [32]:
n_years_in_last_quasi_decade

8

In [33]:
decade_list

['1970', '1980', '1990', '2000', '2010']

In [34]:
#Construct axis labels
ax_labs=pd.Series()
for x in decade_list:
    lab0=x + '-' +x[:-1]+ '9'
    ax_labs=ax_labs.append(pd.Series(lab0))

In [35]:
ax_labs=ax_labs.reset_index(drop=True)
#change first label

ax_labs[0]=str(dat.Precipitation.first_valid_index().year)+ '-' +ax_labs[0].split('-')[1]
ax_labs=ax_labs.values

In [36]:
print('stop: code, assumes that first and last decades incomplete; first is lumped, final is kept separate')

stop: code, assumes that first and last decades incomplete; first is lumped, final is kept separate


In [37]:
#See if there is a trend in these counts of extreme events

#fig, axs = plt.subplots(3,1, sharex=True, sharey=True)

df_list=[hot_temp_dat, cold_temp_dat, high_precip_dat]
df_nmz=['high temperature', 'cold temperature', 'high precipitation']
for ii in range(0, len(df_list)):
    df=df_list[ii]
    df.name=df_nmz[ii]
    var='Temperature'
    if df.equals(high_precip_dat):
        var='Precipitation'

    #Bin early years (not full decade) into the first decade
    df.loc[df.decades==pd.np.nan, 'decades']=df.decades[1]

    #Count occurances in each decade
    count_df=df[[var, 'season', 'decades']].groupby(['season', 'decades']).count()
    count_df.reset_index(inplace=True)
    count_df[var]=count_df[var].fillna(value=0).copy()
    count_df

    #Deal with semi-decades (more or less than 10 yr bins)
    count_df.loc[count_df.decades==decade_list[-1], var]=count_df.loc[count_df.decades==decade_list[-1], var]* (10./n_years_in_last_quasi_decade) #  incomplete decade scaled by the number of years in the decade
    count_df.loc[count_df.decades==decade_list[0], var]=count_df.loc[count_df.decades==decade_list[0], var] *((10.+n_extra_yrs_first_bin)/10)

    count_df['decimal_date']=count_df.decades.astype('float')+5
    #count_df['decades']=count_df['decades'].astype(str)

    
    #count_df=count_df[count_df.decades!= decade_list[0]] #remove 1960s; remains as a remnant of the categorical variable, though has been moved to the 70s

    plt.figure()
    ax=sns.barplot(x="decades", y=var, hue="season", data=count_df, palette='coolwarm')
    plt.ylabel("Scaled number of events")
    plt.Axes.set_xticklabels(ax, labels=ax_labs) #need to exclude first label of the partial decade 
    plt.title("Decadal counts of the top 10 most extreme seasonal " +df.name + " events")
    plt.tight_layout()
    plt.xlabel("")
    plt.savefig(save_dir + 'WeatherExtremes/' +Glacier + Station + '_' + df.name +'_frequency.jpg')
    
    
    #See if any seasonal trends are significant
    for seas in count_df.season.unique():
        lm1 = ols(var+ ' ~ decimal_date', data=count_df[count_df.season==seas]).fit()
        pval=lm1.pvalues['decimal_date']
        if pval<0.1:
            print ("significant: " + seas + " " + df.name+ "; pval= " + str(pval) )
        

<IPython.core.display.Javascript object>

significant: Summer high temperature; pval= 0.00662943284494


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
#look at PDO

In [39]:
PDO_dat=pd.read_csv("Q:\Project Data\GlacierData\WeatherStations\Climate_Indices\PDO_ENSO_Pretty.csv")

In [40]:
PDO_dat.Date=pd.to_datetime(PDO_dat.Date, format='%Y%m')
PDO_dat=PDO_dat.set_index('Date')

In [41]:
PDO_dat.head(1)

,Year,Month,PDO,ENSO
Date,,,,
1948-01-01,1948,1,-0.11,0.26


In [42]:
#Aggregate to Monthly data
mth_dat=pd.DataFrame()
mth_dat['precip']=wx.aggregate_time_with_threshold(dat['Precipitation'], 'MS', func='sum', steps_in_period=30)
mth_dat['min_temp']=wx.aggregate_time_with_threshold(dat['Temperature'], 'MS', func='min', steps_in_period=30)
mth_dat['max_temp']=wx.aggregate_time_with_threshold(dat['Temperature'], 'MS', func='max', steps_in_period=30)
mth_dat['mean_temp']=wx.aggregate_time_with_threshold(dat['Temperature'], 'MS', func='mean', steps_in_period=30)

#Temperature Anomaly
mth_dat['temp_anomaly']=(mth_dat.mean_temp-mth_dat.mean_temp.mean())/mth_dat.mean_temp.std()

mth_dat['month']=mth_dat.index.month

In [43]:
#Combine monthly weather data with PDO data
PDO_dat=mth_dat.merge(PDO_dat, left_index=True, right_index=True, how='outer')
PDO_dat['month']=PDO_dat.index.month

In [44]:
PDO_dat['decimal_date']=PDO_dat.index.year+(PDO_dat.index.dayofyear-1)/365

In [45]:
PDO_dat.head(1)

,precip,min_temp,max_temp,mean_temp,temp_anomaly,month,Year,Month,PDO,ENSO,decimal_date
Date,,,,,,,,,,,
1948-01-01,NaN,NaN,NaN,NaN,NaN,1,1948,1,-0.11,0.26,1948.0


In [46]:
summer_dat=PDO_dat[PDO_dat.month.isin(summer_months)].copy()
winter_dat=PDO_dat[PDO_dat.month.isin(winter_months)].copy()
winter_dat['winter_temp_anomaly']=(winter_dat.mean_temp-winter_dat.mean_temp.mean())/winter_dat.mean_temp.std()

In [47]:
PDO_dat[['PDO', 'ENSO']]['1960':'2018'].rolling(3).mean().plot(colormap='Paired')
plt.savefig(save_dir + 'WeatherExtremes/' +"ENSO and PDO Simple Plot.jpg")

<IPython.core.display.Javascript object>

In [48]:
plot_funcs.OLS_plot('PDO', 'winter_temp_anomaly', dat=winter_dat, aspect=1, title='PDO Correlated with Winter Temperature Anomalies (monthly data)')
plt.tight_layout()

plt.savefig(save_dir + 'WeatherExtremes/' +Glacier + Station + '_CorrelationofPDOAndWinterTemperatureAnomalies.jpg')

<IPython.core.display.Javascript object>

In [49]:
#Add decimal date for regression
PDO_dat['decimal_date']=PDO_dat.index.year+(PDO_dat.index.dayofyear-1)/365

In [50]:
import statsmodels.api as sm
col_x=['decimal_date', 'PDO']
col_y='mean_temp'

y = PDO_dat[col_x]
X = PDO_dat[col_y]
X = sm.add_constant(X)
res = sm.OLS(y, X, missing='drop').fit()
pval=res.pvalues[col_x]
r2=res.rsquared_adj
slope=res.params[col_x]


ValueError: shapes (558,2) and (558,2) not aligned: 2 (dim 1) != 558 (dim 0)

In [ ]:
y = PDO_dat.mean_temp
X = PDO_dat[['decimal_date', 'PDO']]
X = sm.add_constant(X)
model1 = sm.OLS(y, X, missing='drop').fit()
model1.summary()

In [ ]:
y = PDO_dat.mean_temp
X = PDO_dat[['decimal_date']]
X = sm.add_constant(X)
model2 = sm.OLS(y, X, missing='drop').fit()
model2.summary()

In [ ]:
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

m01 = ols('temp_anomaly ~ decimal_date', data=PDO_dat).fit()
m02 = ols('temp_anomaly ~ decimal_date + PDO', data=PDO_dat).fit()
anovaResults = anova_lm(m01, m02)

In [ ]:
anovaResults

In [ ]:
m02.summary()

In [ ]:
m02.params

In [ ]:
m01.params

In [ ]:
#Aggregate to YEARLY for plot
y_dat=pd.DataFrame()
y_dat['precip']=wx.aggregate_time_with_threshold(PDO_dat['precip'], 'AS', func='sum', steps_in_period=12, threshold=0.01)
y_dat['mean_temp']=wx.aggregate_time_with_threshold(PDO_dat['mean_temp'], 'AS', func='mean', steps_in_period=12, threshold=0.1)
y_dat['PDO']=wx.aggregate_time_with_threshold(PDO_dat['PDO'], 'AS', func='mean', steps_in_period=12)
#Temperature Anomaly
y_dat['temp_anomaly']=(y_dat.mean_temp-y_dat.mean_temp.mean())/y_dat.mean_temp.std()
y_dat['precip_anomaly']=(y_dat.precip-y_dat.precip.mean())/y_dat.mean_temp.std()
y_dat['precip_anom_frac_avg']=(y_dat.precip-y_dat.precip.mean())/y_dat.precip.mean()


y_dat['year']=y_dat.index.year

y_dat=y_dat['1968':'2016'] #Subset to only FULL years

In [ ]:
#5-year moving window avg of PDO and temperature anomalies
#plt.figure()
var_cols=['temp_anomaly', 'PDO']
ax=y_dat[var_cols].rolling(5, min_periods=5, center=True).mean().plot(color=['red','blue'])
plt.savefig(save_dir + 'WeatherExtremes/' +Glacier + Station + '_PDOandTempAnomaly_TS_5yrmvngavg.jpg')

In [ ]:
plot_funcs.OLS_plot('PDO', 'precip', dat=y_dat)

In [ ]:
var_cols=['precip_anom_frac_avg', 'PDO']

ax=y_dat[var_cols].rolling(5, min_periods=5, center=True).mean().plot(color=['red','blue'])
ax.legend(["Precipitation Anomaly Fraction of Average", "PDO"])
plt.savefig(save_dir + 'WeatherExtremes/' +Glacier + Station + '_PDOandPrecip_TS_5yrmvngavg.jpg')